# Entrenamiento YOLOv8 - Detección de Anomalías Dentales

Este notebook entrena un modelo YOLOv8 para detectar múltiples anomalías dentales en radiografías panorámicas.

## Presentado por:
- Natalia Moreno Montoya - 2230434
- Liseth Esmeralda Erazo Varela - 2231713
- Valentina Bueno Collazos - 2230556

## Características:
- YOLOv8m (medium) - balance entre precisión y velocidad
- Mixed Precision (FP16) automático para optimización de VRAM
- Data augmentation optimizado para radiografías médicas
- Early stopping y checkpointing para evitar overfitting
- Métricas de detección (mAP, Precision, Recall)
- Visualización completa de resultados

## Clases a Detectar:
Este modelo detecta 14 tipos de anomalías dentales:
1. Cordal
2. Apiñamiento
3. Diente rotado
4. Diastema
5. Zona desdentada (dentula)
6. Tratamiento de conducto
7. Fractura
8. Caries
9. Enanismo radicular
10. Diente retenido
11. Resto radicular
12. Dientes sanos
13. Enanismo denticular
14. Diente supernumerario

## Arquitectura del Modelo:
- **YOLOv8m**: 25.9M parámetros
- **Optimizer**: AdamW (mejor que SGD para datasets pequeños)
- **Learning Rate**: 0.001 → 1e-05 (con warmup de 3 épocas)
- **Batch Size**: 8 (optimizado para 4GB VRAM)
- **Image Size**: 640x640 píxeles

## 1. Instalación y Setup

### Descarga Parcial del Dataset
Se utiliza **sparse checkout** de Git para descargar únicamente la carpeta `proyecto_IA_v2` del repositorio, optimizando tiempo y espacio en disco.

In [ ]:
!mkdir tmp_repo && cd tmp_repo

!git init
!git remote add origin https://github.com/natam226/proyecto_IA.git
!git config core.sparseCheckout true

!echo "proyecto_IA_v2/" >> .git/info/sparse-checkout

!git pull origin main

!mv proyecto_IA_v2 /content/
!cd /content && rm -rf tmp_repo


In [1]:
import sys
import subprocess

try:
    import ultralytics
    print(f"✓ Ultralytics ya instalado (versión {ultralytics.__version__})")
except ImportError:
    print("Instalando ultralytics...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ultralytics"])
    print("✓ Ultralytics instalado")

Instalando ultralytics...
✓ Ultralytics instalado


In [ ]:
import warnings
warnings.filterwarnings('ignore')

from pathlib import Path
import yaml
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2

from ultralytics import YOLO
import time
from datetime import datetime

# Configurar estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✓ Importaciones completadas")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✓ Importaciones completadas


## 2. Verificar GPU y Configuración

In [3]:
print("=" * 80)
print("CONFIGURACIÓN DE GPU - RTX 3050 OPTIMIZADA")
print("=" * 80)

if torch.cuda.is_available():
    print(f"✓ PyTorch version: {torch.__version__}")
    print(f"✓ CUDA version: {torch.version.cuda}")
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✓ VRAM Total: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"✓ VRAM Disponible: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated(0)) / 1e9:.2f} GB")

    # Limpiar cache
    torch.cuda.empty_cache()

    device = 0  # GPU 0
    print(f"\n✓ Dispositivo seleccionado: CUDA:{device}")
else:
    print("⚠️ GPU no disponible - usando CPU (será MUY lento)")
    device = 'cpu'

print("=" * 80)

CONFIGURACIÓN DE GPU - RTX 3050 OPTIMIZADA
✓ PyTorch version: 2.8.0+cu126
✓ CUDA version: 12.6
✓ GPU: Tesla T4
✓ VRAM Total: 15.83 GB
✓ VRAM Disponible: 15.83 GB

✓ Dispositivo seleccionado: CUDA:0


## 3. Verificar Dataset

In [13]:
# Ruta al archivo de configuración del dataset
data_yaml_path = Path('/content/proyecto_IA_v2/data.yml')

if not data_yaml_path.exists():
    print(f"⚠️ ERROR: No se encontró {data_yaml_path}")
else:
    # Leer configuración
    with open(data_yaml_path, 'r', encoding='utf-8') as f:
        data_config = yaml.safe_load(f)

    print("✓ Configuración del dataset:")
    print(f"  • Número de clases: {data_config['nc']}")
    print(f"  • Clases: {data_config['names']}")
    print(f"\n  • Train: {data_config['train']}")
    print(f"  • Val: {data_config['val']}")
    print(f"  • Test: {data_config['test']}")


    # Verificar que existen los directorios
    base_dir = Path('/content/proyecto_IA_v2')

    train_images = list((base_dir / 'train' / 'images').glob('*.png')) + list((base_dir / 'train' / 'images').glob('*.jpg'))
    val_images = list((base_dir / 'val' / 'images').glob('*.png')) + list((base_dir / 'val' / 'images').glob('*.jpg'))
    test_images = list((base_dir / 'test' / 'images').glob('*.png')) + list((base_dir / 'test' / 'images').glob('*.jpg'))

    print(f"\n✓ Imágenes encontradas:")
    print(f"  • Train: {len(train_images)} imágenes")
    print(f"  • Val: {len(val_images)} imágenes")
    print(f"  • Test: {len(test_images)} imágenes")
    print(f"  • Total: {len(train_images) + len(val_images) + len(test_images)} imágenes")

✓ Configuración del dataset:
  • Número de clases: 15
  • Clases: ['cordal', 'apinamiento', 'diente_rotado', 'diastema', 'zona_dentula', 'tratamiento_conducto', 'fractura', 'caries', 'enanismo_radicular', 'diente_retenido', 'resto_radicular', 'dientes_sanos', 'enanismo_denticular', 'enanismo_radicular', 'diente_supernumerario']

  • Train: /content/proyecto_IA_v2/train/images
  • Val: /content/proyecto_IA_v2/val/images
  • Test: /content/proyecto_IA_v2/test/images

✓ Imágenes encontradas:
  • Train: 149 imágenes
  • Val: 10 imágenes
  • Test: 10 imágenes
  • Total: 169 imágenes


## 4. Visualizar Ejemplos del Dataset

In [ ]:
def visualize_yolo_annotations(image_path, label_path, class_names, max_boxes=50):
    """Visualiza una imagen con sus anotaciones YOLO."""
    # Leer imagen
    img = cv2.imread(str(image_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]

    # Leer anotaciones
    if label_path.exists():
        with open(label_path, 'r') as f:
            lines = f.readlines()

        # Dibujar cada bounding box
        for line in lines[:max_boxes]:
            parts = line.strip().split()
            if len(parts) >= 5:
                class_id = int(parts[0])
                x_center, y_center, width, height = map(float, parts[1:5])

                # Convertir de YOLO a pixel coordinates
                x1 = int((x_center - width/2) * w)
                y1 = int((y_center - height/2) * h)
                x2 = int((x_center + width/2) * w)
                y2 = int((y_center + height/2) * h)

                # Dibujar
                color = tuple(np.random.randint(0, 255, 3).tolist())
                cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)

                # Etiqueta
                label = class_names[class_id] if class_id < len(class_names) else f"Class {class_id}"
                cv2.putText(img, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    return img

# Visualizar algunas imágenes de ejemplo
if len(train_images) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.flatten()

    for i, img_path in enumerate(train_images[:4]):
        # Buscar archivo de etiquetas correspondiente
        label_path = Path(str(img_path).replace('/images/', '/labels/').replace('\\images\\', '\\labels\\').replace('.png', '.txt').replace('.jpg', '.txt'))

        # Visualizar
        img_with_boxes = visualize_yolo_annotations(img_path, label_path, data_config['names'])

        axes[i].imshow(img_with_boxes)
        axes[i].set_title(f'Ejemplo {i+1}: {img_path.name}', fontsize=10)
        axes[i].axis('off')

    plt.suptitle('Ejemplos del Dataset con Anotaciones', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ No se encontraron imágenes para visualizar")

## 5. Configuración del Entrenamiento

### Configuración de Hiperparámetros:

#### Parámetros del Modelo:
- **modelo**: yolov8m.pt (medium - 25.9M parámetros)
- **epochs**: 100 épocas con early stopping (patience=20)
- **batch size**: 8 (optimizado para 4GB VRAM)
- **image size**: 640x640

#### Optimización de GPU:
- **Mixed Precision (AMP)**: Activado - Reduce uso de VRAM a la mitad
- **Workers**: 2 - Para carga paralela de datos
- **Device**: CUDA (GPU)

#### Optimizador y Learning Rate:
- **Optimizer**: AdamW (adaptive learning rate)
- **Initial LR**: 0.001
- **Final LR**: 1e-05
- **Warmup**: 3 épocas de calentamiento
- **Weight Decay**: 0.0005

#### Data Augmentation para Radiografías:
- **HSV**: Ajustes mínimos (h=0.015, s=0.2, v=0.2) - radiografías son escala de grises
- **Rotación**: ±10 grados
- **Traslación**: 10% del tamaño de imagen
- **Escala**: ±20%
- **Shear**: 5 grados
- **Flip Horizontal**: 50% (simetría dental)
- **Flip Vertical**: 0% (orientación anatómica fija)
- **Mosaic**: 100% (combina 4 imágenes)
- **Mixup**: 10% (mezcla de imágenes)

Estas técnicas aumentan artificialmente el dataset y mejoran la generalización del modelo.

In [ ]:
CONFIG = {
    # Modelo - YOLO Nano es el más ligero
    'model': 'yolov8m.pt',  # Opciones: yolov8n.pt (nano), yolov8s.pt (small), yolov8m.pt (medium)

    # Dataset
    'data': str(data_yaml_path.absolute()),

    # Entrenamiento 
    'epochs': 100,
    'imgsz': 640,  # Tamaño de imagen
    'batch': 8,    # Batch size 

    # Optimización GPU
    'device': device,
    'workers': 2,   # Workers para data loading
    'amp': True,    # Mixed Precision (FP16) 
    'half': False,  # Se activa automáticamente con amp

    # Guardado y logging
    'project': '../runs/detect',
    'name': 'dental_yolov8n',
    'exist_ok': True,
    'save': True,
    'save_period': 10,  # Guardar checkpoint cada 10 épocas

    # Optimizaciones de entrenamiento
    'optimizer': 'AdamW',  # Mejor que SGD para datasets pequeños
    'lr0': 0.001,          # Learning rate inicial
    'lrf': 0.01,           # Learning rate final (lr0 * lrf)
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 3,
    'warmup_momentum': 0.8,
    'warmup_bias_lr': 0.1,

    # Data augmentation (optimizado para radiografías)
    'hsv_h': 0.015,        # Hue augmentation (muy bajo para rayos X)
    'hsv_s': 0.2,          # Saturation
    'hsv_v': 0.2,          # Value/brightness
    'degrees': 10.0,       # Rotación ±10 grados
    'translate': 0.1,      # Traslación
    'scale': 0.2,          # Escala
    'shear': 5.0,          # Shear
    'perspective': 0.0,    # Sin perspectiva (radiografías son planas)
    'flipud': 0.0,         # No flip vertical (anatomía tiene orientación)
    'fliplr': 0.5,         # Flip horizontal (50% - simetría dental)
    'mosaic': 1.0,         # Mosaic augmentation
    'mixup': 0.1,          # Mixup augmentation
    'copy_paste': 0.0,     # No copy-paste para este caso

    # Early stopping y paciencia
    'patience': 20,        # Parar si no mejora en 20 épocas

    # Validación
    'val': True,
    'plots': True,         # Generar gráficas
    'verbose': True,

    # Métricas
    'conf': 0.25,          # Confidence threshold para predicciones
    'iou': 0.7,            # IoU threshold para NMS
}

print("=" * 80)
print("CONFIGURACIÓN DEL ENTRENAMIENTO - YOLOv8")
print("=" * 80)
print(f"Modelo: {CONFIG['model']}")
print(f"Imagen: {CONFIG['imgsz']}x{CONFIG['imgsz']}")
print(f"Batch size: {CONFIG['batch']}")
print(f"Épocas: {CONFIG['epochs']}")
print(f"Workers: {CONFIG['workers']}")
print(f"Mixed Precision (AMP): {CONFIG['amp']}")
print(f"Device: {CONFIG['device']}")
print(f"Optimizer: {CONFIG['optimizer']}")
print(f"Learning Rate: {CONFIG['lr0']} → {CONFIG['lr0'] * CONFIG['lrf']}")
print(f"Early Stopping Patience: {CONFIG['patience']} épocas")
print("=" * 80)

CONFIGURACIÓN DEL ENTRENAMIENTO - YOLOv8
Modelo: yolov8m.pt
Imagen: 640x640
Batch size: 8
Épocas: 100
Workers: 2
Mixed Precision (AMP): True
Device: 0
Optimizer: AdamW
Learning Rate: 0.001 → 1e-05
Early Stopping Patience: 20 épocas

⚡ Configuración optimizada para RTX 3050 (4GB VRAM)


## 6. Crear Modelo YOLOv8

In [16]:
# Cargar modelo YOLOv8
print(f"Cargando modelo {CONFIG['model']}...")
model = YOLO(CONFIG['model'])

print(f"\n✓ Modelo cargado: {CONFIG['model']}")
print(f"  • Tipo: YOLOv8 Nano (optimizado para velocidad)")
print(f"  • Parámetros: ~{sum(p.numel() for p in model.model.parameters()) / 1e6:.1f}M")
print(f"  • Pre-entrenado: COCO dataset (transferencia de aprendizaje)")
print(f"\n📝 Nota: El modelo se ajustará automáticamente a {data_config['nc']} clases")

Cargando modelo yolov8m.pt...

✓ Modelo cargado: yolov8m.pt
  • Tipo: YOLOv8 Nano (optimizado para velocidad)
  • Parámetros: ~25.9M
  • Pre-entrenado: COCO dataset (transferencia de aprendizaje)

📝 Nota: El modelo se ajustará automáticamente a 15 clases


## 7. Entrenar Modelo 

In [ ]:
print("=" * 80)
print("INICIANDO ENTRENAMIENTO - YOLOv8")
print("=" * 80)
print(f"Fecha/Hora inicio: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 80)
print("\n⏳ Esto puede tomar varias horas...")
print("💡 Tip: No uses el PC para tareas pesadas mientras entrena\n")

start_time = time.time()

# ENTRENAR
results = model.train(
    data=CONFIG['data'],
    epochs=CONFIG['epochs'],
    imgsz=CONFIG['imgsz'],
    batch=CONFIG['batch'],
    device=CONFIG['device'],
    workers=CONFIG['workers'],
    amp=CONFIG['amp'],
    project=CONFIG['project'],
    name=CONFIG['name'],
    exist_ok=CONFIG['exist_ok'],
    save=CONFIG['save'],
    save_period=CONFIG['save_period'],
    optimizer=CONFIG['optimizer'],
    lr0=CONFIG['lr0'],
    lrf=CONFIG['lrf'],
    momentum=CONFIG['momentum'],
    weight_decay=CONFIG['weight_decay'],
    warmup_epochs=CONFIG['warmup_epochs'],
    warmup_momentum=CONFIG['warmup_momentum'],
    warmup_bias_lr=CONFIG['warmup_bias_lr'],
    hsv_h=CONFIG['hsv_h'],
    hsv_s=CONFIG['hsv_s'],
    hsv_v=CONFIG['hsv_v'],
    degrees=CONFIG['degrees'],
    translate=CONFIG['translate'],
    scale=CONFIG['scale'],
    shear=CONFIG['shear'],
    perspective=CONFIG['perspective'],
    flipud=CONFIG['flipud'],
    fliplr=CONFIG['fliplr'],
    mosaic=CONFIG['mosaic'],
    mixup=CONFIG['mixup'],
    copy_paste=CONFIG['copy_paste'],
    patience=CONFIG['patience'],
    val=CONFIG['val'],
    plots=CONFIG['plots'],
    verbose=CONFIG['verbose'],
    conf=CONFIG['conf'],
    iou=CONFIG['iou']
)

end_time = time.time()
total_time = end_time - start_time

print("\n" + "=" * 80)
print("ENTRENAMIENTO COMPLETADO ✅")
print("=" * 80)
print(f"Fecha/Hora fin: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Tiempo total: {total_time/3600:.2f} horas ({total_time/60:.1f} minutos)")
print(f"Tiempo por época: {total_time/CONFIG['epochs']:.1f} segundos")
print("=" * 80)

## 8. Resultados del Entrenamiento

### Métricas Finales Obtenidas:
- **Precision**: 0.4517 - El 45.17% de las detecciones son correctas
- **Recall**: 0.4699 - Se detecta el 46.99% de todas las anomalías
- **mAP50**: 0.4472 - Precisión promedio con IoU de 0.5
- **mAP50-95**: 0.2896 - Precisión promedio con IoU de 0.5 a 0.95
- **Fitness**: 0.2896

### Interpretación:
El modelo YOLOv8m muestra un rendimiento balanceado entre precision y recall. Los resultados indican que:
- El modelo detecta aproximadamente la mitad de las anomalías presentes
- Cuando hace una detección, es correcta en el 45% de los casos
- Hay espacio para mejora, especialmente aumentando el dataset o ajustando hiperparámetros

In [ ]:
# Directorio de resultados
results_dir = Path(CONFIG['project']) / CONFIG['name']
print(f"📁 Directorio de resultados: {results_dir}\n")

# Mostrar métricas finales
print("=" * 80)
print("MÉTRICAS FINALES")
print("=" * 80)

# Las métricas están en results.results_dict
metrics = results.results_dict
print(f"\n📊 Métricas de Detección:")
for key, value in metrics.items():
    if isinstance(value, (int, float)):
        print(f"  • {key}: {value:.4f}")

print("\n" + "=" * 80)

## 9. Visualizar Curvas de Entrenamiento

Las curvas muestran la evolución del entrenamiento a lo largo de las épocas:
- **Pérdidas (Losses)**: box_loss, cls_loss, dfl_loss - deben disminuir
- **Métricas**: Precision, Recall, mAP50, mAP50-95 - deben aumentar
- **Learning Rate**: Muestra el schedule del learning rate con warmup

In [ ]:
# Cargar imagen de resultados generada por YOLO
results_img_path = results_dir / 'results.png'

if results_img_path.exists():
    img = Image.open(results_img_path)

    fig, ax = plt.subplots(figsize=(20, 12))
    ax.imshow(img)
    ax.axis('off')
    ax.set_title('Curvas de Entrenamiento y Métricas - YOLOv8', fontsize=16, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.show()
else:
    print(f"⚠️ No se encontró {results_img_path}")
    print("Las gráficas se generaron en el directorio de resultados.")

## 10. Confusion Matrix

La matriz de confusión normalizada muestra:
- **Diagonal**: Predicciones correctas para cada clase
- **Fuera de diagonal**: Confusiones entre clases
- **Última columna**: Falsos negativos (no detectados)
- **Última fila**: Falsos positivos (detecciones erróneas de fondo)

Esta matriz ayuda a identificar qué clases se confunden entre sí y dónde enfocar mejoras.

In [ ]:
# Cargar matriz de confusión
confusion_matrix_path = results_dir / 'confusion_matrix_normalized.png'

if confusion_matrix_path.exists():
    img = Image.open(confusion_matrix_path)

    fig, ax = plt.subplots(figsize=(14, 12))
    ax.imshow(img)
    ax.axis('off')
    ax.set_title('Matriz de Confusión Normalizada - YOLOv8', fontsize=16, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.show()
else:
    print(f"⚠️ No se encontró matriz de confusión en {confusion_matrix_path}")

## 11. Validar en Test Set

Se evalúa el mejor modelo guardado (best.pt) en el conjunto de test independiente.

### Resultados en Test:
- **Precision**: 0.4548
- **Recall**: 0.3393
- **mAP50**: 0.4197
- **mAP50-95**: 0.2423

El rendimiento en test es similar al de validación, indicando que el modelo generaliza adecuadamente sin overfitting significativo.

In [21]:
print("Validando en test set...\n")

# Cargar mejor modelo
best_model_path = results_dir / 'weights' / 'best.pt'
model_best = YOLO(best_model_path)

# Validar
test_results = model_best.val(
    data=CONFIG['data'],
    split='test',
    imgsz=CONFIG['imgsz'],
    batch=CONFIG['batch'],
    conf=CONFIG['conf'],
    iou=CONFIG['iou'],
    device=CONFIG['device']
)

print("\n" + "=" * 80)
print("RESULTADOS EN TEST SET")
print("=" * 80)
print(f"\n📊 Métricas de Test:")
for key, value in test_results.results_dict.items():
    if isinstance(value, (int, float)):
        print(f"  • {key}: {value:.4f}")
print("\n" + "=" * 80)

Validando en test set...

Ultralytics 8.3.220 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,848,445 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 140.0±27.7 MB/s, size: 2533.9 KB)
val: Scanning /content/proyecto_IA_v2/test/labels... 10 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 10/10 109.7it/s 0.1s
val: New cache created: /content/proyecto_IA_v2/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 1.3it/s 1.5s
                   all         10         49      0.455      0.339       0.42      0.242
                cordal          8         20       0.81       0.85      0.846      0.531
           apinamiento          2          4          1       0.25      0.625      0.312
              diastema          4          5        0.4        0.4       0.45      0.291
          zona_dentula          5         10     

## 12. Ejemplos de Predicciones

In [ ]:
# Hacer predicciones en imágenes de test
if len(test_images) > 0:
    print("Realizando predicciones en imágenes de test...\n")

    # Seleccionar algunas imágenes
    sample_images = test_images[:min(4, len(test_images))]

    # Predecir
    predictions = model_best.predict(
        source=[str(img) for img in sample_images],
        conf=CONFIG['conf'],
        iou=CONFIG['iou'],
        imgsz=CONFIG['imgsz'],
        device=CONFIG['device'],
        save=False,
        verbose=False
    )

    # Visualizar
    fig, axes = plt.subplots(2, 2, figsize=(18, 14))
    axes = axes.flatten()

    for i, (pred, img_path) in enumerate(zip(predictions, sample_images)):
        # Obtener imagen con predicciones
        img_with_pred = pred.plot()  # Dibuja las predicciones
        img_with_pred = cv2.cvtColor(img_with_pred, cv2.COLOR_BGR2RGB)

        axes[i].imshow(img_with_pred)
        axes[i].set_title(f'Predicción {i+1}: {img_path.name}\nDetecciones: {len(pred.boxes)}', fontsize=10)
        axes[i].axis('off')

    plt.suptitle('Predicciones en Test Set - YOLOv8', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

    # Mostrar detalles de detecciones
    print("\n" + "=" * 80)
    print("DETALLES DE DETECCIONES")
    print("=" * 80)
    for i, pred in enumerate(predictions):
        print(f"\nImagen {i+1}: {sample_images[i].name}")
        print(f"  Total de detecciones: {len(pred.boxes)}")

        if len(pred.boxes) > 0:
            for j, box in enumerate(pred.boxes):
                cls_id = int(box.cls[0])
                conf = float(box.conf[0])
                cls_name = data_config['names'][cls_id]
                print(f"    {j+1}. {cls_name}: {conf:.2%} confianza")
        else:
            print("    (Sin detecciones)")
else:
    print("⚠️ No hay imágenes de test disponibles")

## 13. Resumen Final

In [ ]:
print("\n" + "=" * 80)
print("RESUMEN FINAL DEL ENTRENAMIENTO - YOLOv8")
print("=" * 80)

print(f"\n🎯 Resultados:")
print(f"  • mAP50: {test_results.results_dict.get('metrics/mAP50(B)', 0):.4f}")
print(f"  • mAP50-95: {test_results.results_dict.get('metrics/mAP50-95(B)', 0):.4f}")
print(f"  • Precision: {test_results.results_dict.get('metrics/precision(B)', 0):.4f}")
print(f"  • Recall: {test_results.results_dict.get('metrics/recall(B)', 0):.4f}")

print("\n" + "=" * 80)
print("✅ ENTRENAMIENTO YOLOv8 COMPLETADO EXITOSAMENTE")
print("=" * 80)

## 14. Limpiar Memoria GPU

In [ ]:
# Liberar memoria GPU
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("✓ Cache de GPU limpiada")
    print(f"  VRAM disponible: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated(0)) / 1e9:.2f} GB")